# マルコフモデル

まずはシンプルなモデルで言語モデルを学ぶ。

In [1]:
import random
from typing import Dict

import markovify
from datasets import load_dataset
import MeCab


---

## マルコフモデル

*Markov Model*

マルコフ過程に従う確率モデル。マルコフ連鎖とも。マルコフ過程とは、ある状態から次の状態へ遷移する確率がその状態のみに依存する確率過程のこと。次の状態を予測する際に、過去の状態の情報を使わない。

$$
p(x_t | x_{t-1}, x_{t-2}, \ldots, x_1) = p(x_t | x_{t-1})
$$


---

## マルコフモデルを用いた言語モデル

マルコフモデルによる言語モデル（以後マルコフモデルと呼ぶ）では、状態を単語とし、ある単語の次に続く単語の確率を記述する。

自然な文章を生成するためには、前章で作ったようなランダムな出力を行うモデルではなく、入力された文脈を考慮したモデルが必要となる。マルコフモデルはそのようなモデルの中で特に理解しやすいシンプルなモデルである。このモデルは、与えられた文脈の中の最後の単語のみに着目し、次の単語を予測する。

マルコフモデルでは、モデルが持っている全ての語彙に対して、その次に続く単語とその確率を記述する。

例えば、モデルが以下の語彙を持っているとする。

- 今日
- カレー
- 天気
- おいしい
- は
- いい
- 。

これらに対して、次に続く単語とその確率を定める。適当に、主観で決めてみよう。

単語 | 次に続く単語(確率)
--- | ---
今日 | は(1.0)
カレー | は(1.0)
天気 | は(0.5), 。(0.5)
おいしい | 。(0.5), カレー(0.5)
は | 今日(0.25), カレー(0.25), おいしい(0.25), いい(0.25)
いい | 天気(0.5), 。(0.5)
。 |

句点は文の終わりを表すので、次に続く単語はないと考える。

辞書として定義しておこう。

In [2]:
vocab = {
    "今日": {"は": 1},
    "カレー": {"は": 1},
    "天気": {"は": 0.5, "。": 0.5},
    "おいしい": {"。": 0.5, "カレー": 0.5},
    "は": {"今日": 0.25, "カレー": 0.25, "おいしい": 0.25, "いい": 0.25},
    "いい": {"天気": 0.5, "。": 0.5},
}

マルコフモデルは、この確率に基づいて次の単語を予測する。それを繰り返して文章を生成する。

モデルを作ってみよう。

In [3]:
WordProb = Dict[str, float]

class MarkovModel:
    def __init__(self, vocab: Dict[str, WordProb]):
        self.vocab = vocab

    def __call__(self, context):
        last_word = context[-1]
        words, dist = zip(*self.vocab[last_word].items())
        word, = random.choices(words, weights=dist)
        return word


できた。これで文章を生成してみよう。

In [4]:
def generate_sentence(model: MarkovModel, start_word: str):
    context = [start_word]
    next_word = start_word
    while next_word != "。":
        next_word = model(context)
        context.append(next_word)
    return " ".join(context)

初めの単語はこちらで指定する必要がある。

In [5]:
model = MarkovModel(vocab)
print(generate_sentence(model, "今日"))
print(generate_sentence(model, "カレー"))

今日 は おいしい 。
カレー は いい 天気 。


文脈を考慮したモデルが作成できた。次に続く単語に対してまあまあ妥当な確率を手動で設定したので、少しは自然さが見られるはず。


---

## マルコフモデルの学習

語彙と次の単語の確率をデータセットから自動で決定する。

語彙に紐づいている次の単語の確率が、マルコフモデルの全てを表していると見られる。  
前節ではこの確率を手動で定めたが、データを元に自動で設定する事を考える。

このような、モデルの動きを決める値（マルコフモデルの場合は次に続く単語の確率）はパラメータと呼ぶ。そして、適切なパラメータをデータセットから自動で求める事を**機械学習**（や学習）と呼ぶ。

ではやってみよう。

データセットとして、いくつかの単語列を用意する。そして、データセットに出現する全ての単語を対象に、その次に続く単語とその確率を記録する。こうすることでマルコフモデルが完成する。

確率は単語の出現頻度から定める。例として以下の3つの単語列をデータセットとして考えてみよう。

- 今日 は いい 天気 です 。
- 今日 は カレー を 食べ ました 。
- 私 は 今日 カレー を 食べ ました 。
- 私 は カレー が 好き です 。

このデータセットから、例えば、「今日」の次に来る可能性がある単語が「は」と「カレー」であると分かる。出現頻度を見ると、「は」が2回、「カレー」が1回出現しているため、「は」に2/3、「カレー」に1/3の確率を定義できる。このようにして、全ての単語について次に続く単語とその確率を定義する。

実際にやってみよう。

In [6]:
data = [
    "今日 は いい 天気 です 。",
    "今日 は カレー を 食べ ました 。",
    "私 は 今日 カレー を 食べ ました 。",
    "私 は カレー が 好き です 。",
]

In [7]:
vocab = {}
for sent in data:
    sent = sent.split()
    for w1, w2 in zip(sent[:-1], sent[1:]):
        if w1 not in vocab:
            vocab[w1] = {}
        if w2 not in vocab[w1]:
            vocab[w1][w2] = 0
        vocab[w1][w2] += 1

for w1 in vocab:
    total = sum(vocab[w1].values())
    for w2 in vocab[w1]:
        vocab[w1][w2] /= total

vocab

{'今日': {'は': 0.6666666666666666, 'カレー': 0.3333333333333333},
 'は': {'いい': 0.25, 'カレー': 0.5, '今日': 0.25},
 'いい': {'天気': 1.0},
 '天気': {'です': 1.0},
 'です': {'。': 1.0},
 'カレー': {'を': 0.6666666666666666, 'が': 0.3333333333333333},
 'を': {'食べ': 1.0},
 '食べ': {'ました': 1.0},
 'ました': {'。': 1.0},
 '私': {'は': 1.0},
 'が': {'好き': 1.0},
 '好き': {'です': 1.0}}

これで、マルコフモデルの学習が完了したことになる。このモデルでいくつか文章を生成してみよう。

In [8]:
model = MarkovModel(vocab)

for _ in range(10):
    start_word, = random.choices(["今日", "私"]) # 最初の単語はランダム
    print(generate_sentence(model, start_word))

私 は カレー が 好き です 。
私 は 今日 は 今日 は いい 天気 です 。
私 は いい 天気 です 。
今日 カレー を 食べ ました 。
今日 カレー を 食べ ました 。
今日 は 今日 は 今日 カレー を 食べ ました 。
私 は カレー を 食べ ました 。
私 は カレー が 好き です 。
私 は 今日 は いい 天気 です 。
今日 は カレー を 食べ ました 。


ランダムなモデルよりは自然な文章が生成されているのではないだろうか。

これで、データセットからマルコフモデルを学習し、文章を生成することが出来た。

### markovify

マルコフモデルを用いて文章を生成するためのライブラリ。

- [jsvine/markovify: A simple, extensible Markov chain generator.](https://github.com/jsvine/markovify)

これを使うと簡単にマルコフモデルを用いた言語モデルを実装できる。

In [9]:
import markovify

In [10]:
data # 再掲

['今日 は いい 天気 です 。',
 '今日 は カレー を 食べ ました 。',
 '私 は 今日 カレー を 食べ ました 。',
 '私 は カレー が 好き です 。']

In [11]:
model = markovify.Text(data, state_size=1) # 学習

これで学習が完了した。文章を生成してみる。

In [12]:
for _ in range(10):
    sentence = model.make_sentence()
    print(sentence)

今日 は 今日 は 今日 は いい 天気 です 。
None
None
今日 は 今日 は 今日 は カレー を 食べ ました 。
私 は 今日 は いい 天気 です 。
今日 は 今日 カレー が 好き です 。
今日 は 今日 カレー が 好き です 。
私 は 今日 は いい 天気 です 。
私 は 今日 は カレー が 好き です 。
None



---

## より大きなデータセットの活用

ここまで、私が用意した3つの短文をデータセットとして言語モデルを作成した。本節ではもう少し大きなデータセットを用いて言語モデルを作成する。

huggingfaceのdatasetsライブラリを使って、wikipediaの記事をデータセットとして取得する。

In [13]:
from datasets import load_dataset

In [14]:
wiki = load_dataset("wikimedia/wikipedia", "20231101.ja", cache_dir="./data")
wiki

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1389467
    })
})

100万は多いので、適当に1000個の記事を取得する。あと記事全体も多いので、最初の1000文字だけを使う。

In [15]:
data = []
for i in range(100):
    data.append(wiki["train"][i]["text"][:1000])

# Examples
for i in range(3):
    print(data[i][:100].replace("\n", ""))

アンパサンド（&, ）は、並立助詞「…と…」を意味する記号である。ラテン語で「…と…」を表す接続詞 "et" の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、
言語（げんご）は、狭義には「声による記号の体系」をいう。広辞苑や大辞泉には次のように解説されている。人間が音声や文字を用いて思想・感情・意志等々を伝達するために用いる記号体系。およびそれを用いる
日本語（にほんご、にっぽんご）は、日本国内や、かつての日本領だった国、そして国外移民や移住者を含む日本人同士の間で使用されている言語。日本は法令によって公用語を規定していないが、法令その他の公用文は全


### トークン化

テキストをトークンごとに分割する。トークンとはモデルが扱う最小単位のことで、例えば単語が該当する。本節でも単語を最小単位=トークンとして、文章を単語列に変換する。

日本語の文章は単語間にスペースが入っていないので、こちらで分割する必要がある。ここには形態素解析器を使用する。これは自然言語処理で形態素解析（品詞の分析）に使用するツール。色々な種類があるが、ここでは[janome](https://mocobeta.github.io/janome/)を使用する。

In [16]:
from janome.tokenizer import Tokenizer

In [17]:
tokenizer = Tokenizer()
def tokenize(text):
    return " ".join(tokenizer.tokenize(text, wakati=True))

tokenize("私は猫が好きです。")

'私 は 猫 が 好き です 。'

これを使って、学習データの全てを単語ごとに分割する。

In [18]:
data_wakati = []
for sentence in data:
    data_wakati.append(tokenize(sentence))

# Examples
for i in range(3):
    print(data_wakati[i][:100].replace("\n", ""))

アンパサンド （&,   ） は 、 並立 助詞 「 … と … 」 を 意味 する 記号 で ある 。 ラテン語 で 「 … と … 」 を 表す 接続詞   " et "   の 合 字 を 起源
言語 （ げん ご ） は 、 狭義 に は 「 声 による 記号 の 体系 」 を いう 。  広辞苑 や 大辞泉 に は 次 の よう に 解説 さ れ て いる 。  人間 が 音声 や 
日本語 （ に ほん ご 、 にっぽん ご ） は 、 日本 国内 や 、 かつて の 日本 領 だっ た 国 、 そして 国外 移民 や 移住 者 を 含む 日本人 同士 の 間 で 使用 さ れ 


### 実践

markovifyを使ってマルコフモデルを学習させ、文章を生成する。

In [19]:
model = markovify.NewlineText(data_wakati, state_size=1)

In [20]:
for _ in range(5):
    sentence = model.make_sentence(max_words=100).replace(" ", "")
    print(sentence)

メリーゴーランド、ラ・スペツィア＝リミニ線・システム
古代エジプト。よってS→T/RPM形式
もっとも古くから構成されたほか、他にはトランス
政治関係に法的にリビア、絵画や背景にギリシア
政治などである。また、他の宗教などが、チューリング認識可能性は西に据え付けても落選、昼間は多数の河川、


語彙が増えたことで多様な文章が生成されるようになった。

ただ、文脈の最後の単語しか考慮できないので、文章全体での自然さを作り出すことは難しい。


---

## N階マルコフモデル

直前のN単語を考慮するマルコフモデル。N=1の場合は通常のマルコフモデルとなる。

学習はこれまでと同様。N個の単語を1つの単位として次に続く単語の確率分布を求める。以下のデータセット:

- 今日 は いい 天気 です 。
- 今日 は カレー を 食べ ました 。
- 私 は 今日 カレー を 食べ ました 。
- 私 は カレー が 好き です 。

から、N=2で確率分布を求めると:

- 「今日 は」の次は「いい」が0.5、「カレー」が0.5
- 「は いい」の次は「天気」が1
- 「いい 天気」の次は「です」が1
- ...

て感じ。

markovifyで実装してみる。`state_size`でNを指定する。N=2としてみよう（3以上だと上手く生成できなかった。）

In [21]:
model = markovify.NewlineText(data_wakati, state_size=2)

In [22]:
for _ in range(5):
    sentence = model.make_sentence(max_words=100)
    if sentence:
        sentence = sentence.replace(" ", "")
    print(sentence)

音楽に関係する具体的ジェイオーク人物については、宇宙人、バビロニア人になった。
初期ノード設定等に関しては当時の少女漫画の前衛的なピアノは88鍵を備え、音域が非常にシリアスなストーリーも盛り込まれている。特に、チューリング完全である。
地球物理学の研究対象には含まれており、
学術的な腱鞘炎などの影響を与えた。
言語学のことをその存在様式を研究対象には顔を出さないので、言語に含まれる一方、その発明の産業上利用可能性、進歩性といった特許要件について特許庁による審査を経て10月号から10月号に永森裕二原作の『宗教生活の原初形態』や若い女性向けのゲームタイトル一覧


一応、ほんの少しは自然さが増したハズ。